In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.impute import SimpleImputer 

# Carregar os dados (assumindo que os dados estão em um arquivo CSV)
data = pd.read_csv('/home/augusto/projects/kaggle/cibmtr/data/equity-post-HCT-survival-predictions/train.csv') 

# Separar variáveis preditoras (features) e variável alvo (target)
X = data.drop(['efs', 'efs_time'], axis=1)  # Features
y = data['efs']  # Target

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criar pipelines para transformação de dados
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['float64', 'int64']).columns

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Criar o pipeline do modelo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', (random_state=42))
])

# Treinar o modelo
pipeline.fit(X_train, y_train)

# Fazer previsões
y_pred = pipeline.predict(X_test)

# Avaliar o modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy}")
print(classification_report(y_test, y_pred))

# Salvar o modelo (opcional)
# import joblib
# joblib.dump(pipeline, 'modelo_efs.joblib') 

# Carregar o modelo (opcional)
# loaded_model = joblib.load('modelo_efs.joblib')

Acurácia: 0.6744791666666666
              precision    recall  f1-score   support

         0.0       0.68      0.56      0.62      2683
         1.0       0.67      0.77      0.72      3077

    accuracy                           0.67      5760
   macro avg       0.68      0.67      0.67      5760
weighted avg       0.68      0.67      0.67      5760



In [4]:
# Treinar o modelo
pipeline.fit(X, y)

X_test = pd.read_csv('data/equity-post-HCT-survival-predictions/test.csv')
y_pred = pipeline.predict(X_test)

# Create the submission: ID, prediction
submission = pd.DataFrame({
    'ID': X_test.ID, 
    'prediction': y_pred
})

submission.to_csv('submission.csv', index = False)